In [5]:
%load_ext gams_magic

The gams_magic extension is already loaded. To reload it, use:
  %reload_ext gams_magic


Sunco Oil manufactures three types of gasoline (gas 1, gas 2, gas 3) by blending three types of crude oil 
(crude 1, crude 2 and crude 3). When different types of crude oil are mixed, their volumes add up, and 
their octane levels and sulfur content blend linearly.

|Crude|Purchase price|Octane level|Sulfur content|
|-----|--------------|------------|--------------|
|1    |45\$/Barrel   |12          |0.5           |
|2    |35\$/Barrel   |6           |2             |
|3    |25\$/Barrel   |8           |3             |

|Gas|Selling price|Octane level|Sulfur content|Demand (barrels)|
|---|-------------|------------|--------------|----------------|
|1  |70\$/Barrel  |>= 10       |<=1           |3000            |
|2  |60\$/Barrel  |>= 8        |<=2           |2000            |
|3  |50\$/Barrel  |>= 6        |<=1           |1000            |

Each day, Sunco can purchase up to 5000 barrels of each type of crude oil, and can produce up to 14000 barrels 
of gasoline totally. It costs \$4 to transform one barrel of oil into one barrel of gasoline. Assume that Sunco
can sell all gasoline it produces. How does it maximize daily profit while satisfying the demands?

In [6]:
%%gams

Set
    i 'crude' / c1, c2, c3 /,
    j 'gas' / g1, g2, g3 /,
    k 'characteristics' / price, octane, sulfur /;
    
Table crude(i,k) 'types of crude and their characteristics'
        price octane sulfur
    c1   45      12      0.5
    c2   35      6       2
    c3   25      8       3;

Table gas(j,k) 'types of gas and their characteristics'
        price octane sulfur
    g1   70      10      1
    g2   60      8       2
    g3   50      6       1;
  
Parameter d(j) 'Demand (barrels)' / g1  3000,
                                    g2  2000,
                                    g3  1000 /;
                                    
*Sunco can purchase up to 5000 barrels of each type of crude oil
parameter crude_max  /5000/;

*can produce up to 14000 barrels of gasoline totally
parameter gas_max  /14000/;

*It costs $4 to transform one barrel of oil into one barrel of gasoline
parameter oil_to_gas  /4/;

Variables
    x(i,j)    'inmput of i used to produce j'
    profit    'total cost';
   
Positive Variable x;  

Equation
    obj         'calculate profit of the gas production and the crude cost'
    octane(j)   'octane level of gas j must achieve a minimum level'
    sulfur(j)   'sulfur level of gas j must achieve a maximum level'
    c_max(i)    'up to 5000 (crude_max) barrels of each type of crude oil'
    g_max       'up to 14000 (gas_max) barrels of gasoline totally'
    demand(j)   'the production of each gas j must satisfy the demands'
    ;
    
    obj.. sum((i,j), x(i,j)*gas(j,'price') - x(i,j)*crude(i,'price') - x(i,j)*oil_to_gas )=e= profit;
    octane(j).. sum(i,x(i,j)*crude(i,'octane')) =g= gas(j,'octane')*sum(i,x(i,j));
    sulfur(j).. sum(i,x(i,j)*crude(i,'sulfur')) =l= gas(j,'sulfur')*sum(i,x(i,j));
    c_max(i).. sum(j,x(i,j)) =l= crude_max;
    g_max.. sum((i,j),x(i,j)) =l= gas_max;
    demand(j).. sum(i,x(i,j)) =g= d(j);

model prob /all/;
Options LP  = Cplex;
Solve prob using LP maximizing profit;
display profit.l, x.l;


,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),288500.0,14,10,LP,CPLEX,0.078


In [7]:
%gams_lst

GAMS 28.2.0  r750fa45 Released Aug 19, 2019 WEX-WEI x86 64bit/MS Windows                                                                                                                                                              09/30/19 15:19:29 Page 10
G e n e r a l   A l g e b r a i c   M o d e l i n g   S y s t e m
C o m p i l a t i o n


  66   
  67  Set
  68      i 'crude' / c1, c2, c3 /,
  69      j 'gas' / g1, g2, g3 /,
  70      k 'characteristics' / price, octane, sulfur /;
  71   
  72  Table crude(i,k) 'types of crude and their characteristics'
  73          price octane sulfur
  74      c1   45      12      0.5
  75      c2   35      6       2
  76      c3   25      8       3;
  77   
  78  Table gas(j,k) 'types of gas and their characteristics'
  79          price octane sulfur
  80      g1   70      10      1
  81      g2   60      8       2
  82      g3   50      6       1;
  83   
  84  Parameter d(j) 'Demand (barrels)' / g1  3000,
  85                              

In [8]:
# you may need to modify the following variable names
%gams display x.l, profit.l;
%gams_lst -e

E x e c u t i o n


----    125 VARIABLE x.L  inmput of i used to produce j

            g1          g2          g3

c1    2222.222    2111.111     666.667
c2     444.444    4222.222     333.333
c3     333.333    3166.667


----    125 VARIABLE profit.L              =   288500.000  total cost


